In [1]:
from pyspark.sql.functions import col

# ============================================================
# 0. Configurações (Modo Cirúrgico - Manual)
# ============================================================
spark.conf.set("spark.microsoft.delta.optimizeWrite.enabled", "false")
spark.conf.set("spark.microsoft.delta.autoCompact.enabled", "false")
spark.conf.set("spark.sql.adaptive.enabled", "true")

path_destino = (
    "abfss://ws_departamento_pessoal@onelake.dfs.fabric.microsoft.com/"
    "lk_departamento_pessoal.Lakehouse/Tables/tab_gold_dim_funcionario_ultima_posicao_mes"
)

# ============================================================
# 1. Ler e Filtrar
# ============================================================
df_historico = spark.table("tab_gold_dim_funcionario_historico")
df_filtrado = df_historico.filter(col("RK_MES") == 1)

# ============================================================
# 2. ESTRATÉGIA FUNIL (Coalesce 1)
# ============================================================
df_otimizado = df_filtrado.coalesce(1)

# ============================================================
# 3. Contagem e Escrita
# ============================================================
# Realizamos a contagem para log (isso dispara uma ação leve no Spark)
qtd_linhas = df_otimizado.count()

# Escrevemos no disco
(
    df_otimizado.write
        .format("delta")
        .mode("overwrite")
        .option("overwriteSchema", "true")
        .save(path_destino)
)

print(f"Sucesso! Tabela Gold atualizada.")
print(f"Total de registros gravados (Última Posição): {qtd_linhas}")

StatementMeta(, 85698e23-f90e-4d77-8857-1584b16cbcbc, 3, Finished, Available, Finished)

Sucesso! Tabela Gold atualizada.
Total de registros gravados (Última Posição): 827291
